## Setup

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g4H9w2nD1xjJx65G2y8PoduTrUnDlDUylapiKOwOoERocg0XT_4lCA



Successfully saved authorization token.


In [213]:
import numpy as np
import geetools
from geetools import ui, cloud_mask
import os, datetime
import config as cf
import pandas as pd

mask_l8SR_all = cloud_mask.landsatSR()

In [214]:
SURVEY_NAME = 'DHS'

## Functions

In [219]:
def survey_to_fc_buffer(survey_df, buffer_size_urban, buffer_size_rural):
    '''
    Convert pandas dataframe of survey locations to a feature collection. 
    
    Inputs:
        survey_df: pandas dataframe of survey locations. Function assumes 
                   the dataframe contains (1) latitude, (2) longitude and
                   (3) uid variables. Assumes coordinates in WGS84.
    Returns:
        (feature collection)
    '''
    
    survey_fc_list = []
    
    n_rows = survey_df.shape[0]
    for i in range(0, n_rows):
        survey_df_i = survey_df.iloc[[i]]
        
        ur = survey_df_i['urban_rural'].iloc[0]
        if ur == 'U':
            buffer_size = buffer_size_urban
        elif ur == 'R':
            buffer_size = buffer_size_rural

        f_i = ee.Feature(ee.Geometry.Point([survey_df_i['longitude'].iloc[0], 
                                            survey_df_i['latitude'].iloc[0]]), 
                         {'uid': survey_df_i['uid'].iloc[0]})
        
        f_i = f_i.buffer(buffer_size)

        survey_fc_list.append(f_i)
        
    survey_fc = ee.FeatureCollection(survey_fc_list)
    
    return survey_fc

def extract_sat(survey_df, buffer_size_urban, buffer_size_rural, satellite):
    '''
    Extract satellite imagery to locations 
    
    Inputs:
        survey_df: pandas dataframe of survey locations. Function assumes 
                   the dataframe contains (1) latitude, (2) longitude and
                   (3) uid variables. Assumes coordinates in WGS84.
    Returns:
        (feature collection)
    '''
    
    
    # Prep satellite
    if satellite == 'l8':
        
        SCALE = 30
        
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
            .filterDate('2017-01-01', '2020-12-31')\
            .map(mask_l8SR_all)\
            .median()\
            .multiply(0.0001)

        ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
        image = image.addBands(ndvi)
        
        BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'NDVI']
        
    # Prep survey
    survey_fc = survey_to_fc_buffer(survey_df, buffer_size_urban, buffer_size_rural)
    
    # Extract values
    vals = image.reduceRegions(collection = survey_fc,
                               reducer = ee.Reducer.mean(),
                               scale = 30,
                               tileScale = 8)
    
    survey_df = survey_df[['uid']]
    
    for band_i in BANDS:
        #print(band_i)
        survey_df[satellite + '_' + band_i] = vals.aggregate_array(band_i).getInfo()
        
    return survey_df

## Load/Prep Survey Data

In [221]:
survey_df = pd.read_csv(os.path.join(cf.SECURE_DATA_DIRECTORY, 'Data', SURVEY_NAME, 'FinalData - PII', 'GPS_uid_crosswalk.csv'))
survey_df = survey_df[survey_df.most_recent_survey == True]
survey_df = survey_df.head(30)
CHUNK_SIZE = 10

In [222]:
survey_df['chunk_id'] = chunk_ids(survey_df.shape[0], CHUNK_SIZE)

## Extract Landsat 8

In [ ]:
vals_df_list = []
for chunk_i in list(np.unique(survey_df.chunk_id)):
    print(chunk_i)
    
    survey_df_i = survey_df[survey_df['chunk_id'] == chunk_i]
    vals_i_df = extract_sat(survey_df_i, 2000, 5000, 'l8')
    
    vals_df_list.append(vals_i_df)
    
vals_df = pd.concat(vals_df_list)

In [223]:
vals_df.to_csv(os.path.join(cf.DROPBOX_DIRECTORY, 
                            'Data', 
                            SURVEY_NAME, 
                            'FinalData', 
                            'Individual Datasets',
                           'survey_l8.csv'),
              index = False)

0
B1


/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


B2
B3
B4
B5
B6
B7
B10
B11
NDVI
1
B1
B2
B3
B4
B5
B6
B7
B10
B11
NDVI
2
B1
B2
B3
B4
B5
B6
B7
B10
B11
NDVI


,uid,l8_B1,l8_B2,l8_B3,l8_B4,l8_B5,l8_B6,l8_B7,l8_B10,l8_B11,l8_NDVI
1919,BD201800000001,0.053339,0.066258,0.094288,0.093794,0.222029,0.175113,0.112021,0.296733,0.295017,0.374844
1920,BD201800000002,0.039662,0.052849,0.088150,0.084419,0.251872,0.197020,0.122449,0.297323,0.295375,0.501263
1921,BD201800000003,0.045617,0.058071,0.089661,0.080594,0.239760,0.162204,0.092258,0.296603,0.294675,0.448543
1922,BD201800000004,0.035296,0.047997,0.080312,0.071710,0.265296,0.183386,0.105234,0.297098,0.294949,0.576071
1923,BD201800000005,0.040190,0.052762,0.084024,0.075013,0.249323,0.166932,0.095310,0.296479,0.294688,0.504198
1924,BD201800000006,0.049760,0.064042,0.095932,0.092262,0.224155,0.159823,0.097206,0.296592,0.294775,0.355252
1925,BD201800000007,0.027753,0.037215,0.066813,0.050193,0.244763,0.134475,0.067784,0.295683,0.293518,0.655736
1926,BD201800000008,0.037656,0.050128,0.082874,0.070129,0.243935,0.158568,0.088632,0.296148,0.294044,0.516463
1927,BD201800000009,0.041914,0.053410,0.085631,0.075182,0.229667,0.158258,0.092185,0.296264,0.294622,0.460216
1928,BD201800000010,0.036150,0.048188,0.079740,0.069044,0.254635,0.167873,0.094076,0.296326,0.294601,0.548530


In [193]:
#import folium
#from folium import plugins

#mapid = survey_i_fc.getMapId()

#map = folium.Map(location=[80.773137, 7.873592])
#folium.TileLayer(
#    tiles=mapid['tile_fetcher'].url_format,
#    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#    overlay=True,
#    name='border',
#  ).add_to(map)

#map.add_child(folium.LayerControl())
